In [ ]:
%matplotlib widget

In [ ]:
from pathlib import Path
import numpy as np
from bouter import EmbeddedExperiment
import pandas as pd
from tqdm import tqdm
import flammkuchen as fl

from matplotlib import pyplot as plt
import seaborn as sns
sns.set(palette="deep", style="ticks")
cols = sns.color_palette()

from scipy.stats import ranksums, kstest, ttest_ind
from utilities import stimulus_df_from_exp

In [ ]:
pooled_dicts = fl.load("/Users/luigipetrucco/Desktop/pooled_resps.h5")

In [ ]:
fids = list(pooled_dicts.keys())

In [ ]:
from numba import njit, prange

@njit
def roll_matrix(input_mat, indexes):
    
    output_mat = np.empty_like(input_mat)
    
    for i in prange(output_mat.shape[1]):
        output_mat[:, i] = np.roll(input_mat[:, i], indexes[i])
    
    return output_mat

def center_on_peak(input_mat):
    """Recenter along the 1st dimension.
    """
    idxs = - np.argmax(input_mat, 0) - input_mat.shape[0] // 2
    
    return roll_matrix(input_mat, idxs)

In [ ]:
master_path = Path("/Volumes/Shared/experiments/E0070_receptive_field/v04_flashing_rad_simple")
path = master_path / "210611_f5"
responses = pd.DataFrame(fl.load(path / "cell_resps.h5", "/rel_scores"))

# Load traces and experiment metadata:
coords = fl.load(path / "data_from_suite2p_unfiltered.h5", "/coords")
rois = fl.load(path / "data_from_suite2p_unfiltered.h5", "/rois_stack")
mask = fl.load(path / "anatomy.mask", "/mask")
responses = pd.DataFrame(fl.load(path / "cell_resps.h5", "/rel_scores")).values
in_tectum = pd.DataFrame(fl.load(path / "cell_resps.h5", "/in_tectum"))

exp = EmbeddedExperiment(path)
stim_df = stimulus_df_from_exp(exp)

responsive = responses.max(0) > 0.4
all_peaks = np.argmax(responses, 0)

In [ ]:
master_path = Path("/Volumes/Shared/experiments/E0070_receptive_field/v04_flashing_rad_simple")
path = master_path / "210611_f5"
exp = EmbeddedExperiment(path)
stim_df = stimulus_df_from_exp(exp)

stim_thetas = np.array(sorted(stim_df["pos_start"].unique()))
l_r_position = np.cos(stim_thetas)
a_p_position = -np.sin(stim_thetas)

# All cells

In [ ]:
pooled_data = fl.load(master_path / "new_pooled.h5")

In [ ]:
responses = pooled_data.loc[:, [i for i in range(35)]].values.T
coords = pooled_data.loc[:, ["z", "x", "y"]].values
in_tectum = pooled_data["in_tectum"].values

responsive = responses.max(0) > 0.5
all_peaks = np.argmax(responses, 0)

In [ ]:
f, all_axs = plt.subplots(2, 2, figsize=(7, 4))

xaxis = np.linspace(-np.pi, np.pi, 100)
yaxis = np.linspace(-np.pi, np.pi, 100)

axs = all_axs[:, 0]
for i in range(9):
    filt = responsive & in_tectum & (coords[:, 0] == i) 
    axs[0].scatter(coords[filt, 1], coords[filt, 2], 
                c=stim_thetas[all_peaks[filt]], cmap="twilight_shifted", s=8)
axs[0].axis("equal")
axs[0].axis("off")

for i in range(12, 0, -1):
    filt = responsive & in_tectum & (coords[:, 2] > i*50) & (coords[:, 2] < (i+1)*50)
    axs[1].scatter(coords[filt, 1], coords[filt, 0]*10, 
                c=stim_thetas[all_peaks[filt]], cmap="twilight_shifted", s=8)
axs[1].axis("equal")
axs[1].axis("off")

axs = all_axs[:, 1]
filt = responsive & in_tectum

axs[0].scatter(coords[filt, 1] * 0.6, -np.cos(stim_thetas[all_peaks[filt]]), s=5)
axs[0].set_xlabel("Left - Right position (um)")
axs[0].set_ylabel("cos(θ)")
axs[0].set_ylim(-1.5, 1.5)

axs[1].scatter(coords[filt, 2] * 0.6, np.sin(stim_thetas[all_peaks[filt]]), s=5)
axs[1].set_xlabel("Posterior - Anterior position (um)")
axs[1].set_ylabel("sin(θ)")
axs[1].set_ylim(-1.5, 1.5)
sns.despine()
plt.tight_layout()

ax = f.add_axes((0.2, 0.45, 0.1, 0.1))
ax.imshow(np.angle(yaxis[:,None] + 1j*xaxis[None,:]).T, 
          cmap="twilight_shifted", extent=[0, 1, 0, 1])
ax.text(0,1.1, "stim. θ")
ax.axis("equal")
ax.axis("off")

In [ ]:
f = plt.figure()
ax = plt.subplot(111, polar=True)

ax.bar(stim_thetas, c, lw=0, width=np.pi/18)

ax.set_thetagrids([])

ax.spines['polar'].set_visible(False) 
#ax.set_ylim(0, 1)
ax.set_rgrids([0.3, 0.6, 0.9], angle=0, fontsize=9)
ax.text(-0.1, 0.68,
        'Reliability', rotation=0,ha='center',va='center', fontsize=10)

In [ ]:
f, all_axs = plt.subplots(2, 3, figsize=(7, 4))

xaxis = np.linspace(-np.pi, np.pi, 100)
yaxis = np.linspace(-np.pi, np.pi, 100)

for g_i, g in enumerate(["Huc:H2B-GCaMP6s", "Huc:H2B-GCaMP6s;olig1:Ntr"]):
    axs = all_axs[:, g_i]
    for i in range(9):
        filt = responsive & in_tectum & (coords[:, 0] == i) & (pooled_data["gen"] == g)
        axs[0].scatter(coords[filt, 1], coords[filt, 2], 
                    c=stim_thetas[all_peaks[filt]], cmap="twilight_shifted", s=8)
    axs[0].axis("equal")
    axs[0].axis("off")
    axs[0].set_title(g)

    for i in range(12, 0, -1):
        filt = responsive & in_tectum & (pooled_data["gen"] == g) & (coords[:, 2] > i*50) & (coords[:, 2] < (i+1)*50)
        axs[1].scatter(coords[filt, 1], coords[filt, 0]*10, 
                    c=stim_thetas[all_peaks[filt]], cmap="twilight_shifted", s=8)
    axs[1].axis("equal")
    axs[1].axis("off")

axs = all_axs[:, 2]
for g_i, g in enumerate(["Huc:H2B-GCaMP6s", "Huc:H2B-GCaMP6s;olig1:Ntr"]):
    filt = responsive & in_tectum & (pooled_data["gen"] == g)

    axs[0].scatter(coords[filt, 1] * 0.6, np.cos(stim_thetas[all_peaks[filt]]), alpha=0.4, s=5)
    axs[0].set_xlabel("Left - Right position (um)")
    axs[0].set_ylabel("cos(θ)")
    axs[0].set_ylim(-1.5, 1.5)

    axs[1].scatter(coords[filt, 2] * 0.6, np.sin(stim_thetas[all_peaks[filt]]), alpha=0.4, s=5)
    axs[1].set_xlabel("Posterior - Anterior position (um)")
    axs[1].set_ylabel("sin(θ)")
    axs[1].set_ylim(-1.5, 1.5)
    sns.despine()
    plt.tight_layout()

    ax = f.add_axes((0.2, 0.45, 0.1, 0.1))
    ax.imshow(np.angle(yaxis[:,None] + 1j*xaxis[None,:]).T, 
              cmap="twilight_shifted", extent=[0, 1, 0, 1])
    ax.text(0,1.1, "stim. θ")
    ax.axis("equal")
    ax.axis("off")

In [ ]:
f = plt.figure(figsize=(7, 4))
anat_scatt_size=5

all_axs = [[f.add_axes((0.1+0.4*i, 0.1+0.4*j, 0.4, 0.4)) for j in range(1, -1, -1)] for i in range(2)]

xaxis = np.linspace(-np.pi, np.pi, 100)
yaxis = np.linspace(-np.pi, np.pi, 100)

for g_i, g in enumerate(["Huc:H2B-GCaMP6s", "Huc:H2B-GCaMP6s;olig1:Ntr"]):
    axs = all_axs[g_i]
    for i in range(9):
        filt = responsive & in_tectum & (coords[:, 0] == i) & (pooled_data["gen"] == g)
        axs[0].scatter(coords[filt, 1], coords[filt, 2], 
                    c=stim_thetas[all_peaks[filt]], cmap="twilight_shifted", s=anat_scatt_size)
    axs[0].axis("equal")
    axs[0].axis("off")
    axs[0].set_title(g)

    for i in range(12, 0, -1):
        filt = responsive & in_tectum & (pooled_data["gen"] == g) & (coords[:, 2] > i*50) & (coords[:, 2] < (i+1)*50)
        axs[1].scatter(coords[filt, 1], coords[filt, 0]*10, 
                    c=stim_thetas[all_peaks[filt]], cmap="twilight_shifted", s=anat_scatt_size)
    axs[1].axis("equal")
    axs[1].axis("off")
    
ax = f.add_axes((0.1, 0.9, 0.1, 0.1))
ax.imshow(np.angle(yaxis[:,None] + 1j*xaxis[None,:]).T, 
          cmap="twilight_shifted", extent=[0, 1, 0, 1])
ax.text(0,1.1, "stim. θ")
ax.axis("equal")
ax.axis("off")

In [ ]:
responsive

In [ ]:
check_path = Path("/Users/luigipetrucco/Desktop/E0070_rf_estimation/v05_flashing_rad_simple_2dims/210719_f0")
exp = EmbeddedExperiment(check_path)

In [ ]:
logs = exp["stimulus"]["log"][1::2]
all_positions = []
for log in logs:
    clip = log["clip_mask"]

    pos_start = np.arctan2(clip[1][1] - 0.5, clip[1][0] - 0.5)
    pos_end = np.arctan2(clip[2][1] - 0.5, clip[2][0] - 0.5)
    
    all_positions.append(pos_start)

In [ ]:
pos_start

In [ ]:
all_positions

In [ ]:
plt.figure()
plt.scatter(np.cos(all_positions[:4]), -np.sin(all_positions[:4]), c=np.arange(4))

In [ ]:
sw, w, se, n, nw, nw, e, w, e, nw, n, nw, nem sw, e 